In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
import pandas as pd
train = pd.read_csv("/Users/admin/Desktop/database/diabetes/diabetes_train.txt",header=None,index_col=False)
test = pd.read_csv("/Users/admin/Desktop/database/diabetes/diabetes_test.txt",header=None,index_col=False)

In [22]:
def judge(x):
    s=[]
    for i in range(len(x)):
        if x[i]>=0.5:
            y= 1
        else:
            y= 0
        s.append(y)
    return s

In [3]:
label = train.loc[:,[8]].values.reshape(-1)
data = train.drop(columns=8).values.reshape(-1,8)

y_test =  test.loc[:,[8]].values.reshape(-1)
X_test =  test.drop(columns=8).values.reshape(-1,8)

### 随机森林

In [40]:
rf = RandomForestRegressor(n_estimators=1000)  
rf=rf.fit(data, label)  
predictions_rf=rf.predict(X_test)

In [41]:
pred_rf=judge(predictions_rf)

In [42]:
acc = accuracy_score(y_test, pred_rf)
print("Test Accuracy of rf = {:.2f} %".format(acc * 100))

Test Accuracy of rf = 80.97 %


### 极端随机森林

In [27]:
from sklearn.ensemble import  ExtraTreesRegressor

In [28]:
etr=ExtraTreesRegressor(n_estimators=1000)  
etr=etr.fit(data, label)  

In [43]:
predictions_etr=etr.predict(X_test)

In [44]:
pred_etr=judge(predictions_etr)

In [45]:
acc = accuracy_score(y_test, pred_etr)
print("Test Accuracy of pred_etr = {:.2f} %".format(acc * 100))

Test Accuracy of pred_etr = 79.85 %


### 深度森林

In [13]:
import joblib
from gcforest.gcforest import GCForest

In [38]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0  # 0 or 1
    ca_config["n_cascadeRFtree"] = 1000
    ca_config["max_layers"] = 100 # 最大的层数，layer对应论文中的level
    ca_config["early_stopping_rounds"] = 3 #如果出现某层的三层以内的准确率都没有提升，层中止
    ca_config["n_classes"] = 2 #判别的类别数量
    ca_config["estimators"] = []
    
    ca_config["estimators"].append({"n_folds": 2, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 2, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 2, "type": "LogisticRegression"})
    config["cascade"] = ca_config #共使用了3个基学习器
    return config
    
config=get_toy_config()
gc = GCForest(config)
#X_train_enc是每个模型最后一层输出的结果，每一个类别的可能性
X_train_enc = gc.fit_transform(data, label)

[ 2020-06-22 11:26:14,357][cascade_classifier.fit_transform] X_groups_train.shape=[(500, 8)],y_train.shape=(500,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2020-06-22 11:26:14,360][cascade_classifier.fit_transform] group_dims=[8]
[ 2020-06-22 11:26:14,362][cascade_classifier.fit_transform] group_starts=[0]
[ 2020-06-22 11:26:14,364][cascade_classifier.fit_transform] group_ends=[8]
[ 2020-06-22 11:26:14,365][cascade_classifier.fit_transform] X_train.shape=(500, 8),X_test.shape=(0, 8)
[ 2020-06-22 11:26:14,366][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(500, 8), X_cur_test.shape=(0, 8)
[ 2020-06-22 11:26:14,591][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2_folds.train_0.predict)=71.60%
[ 2020-06-22 11:26:14,813][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2_folds.train_1.predict)=70.40%
[ 2020-06-22 11:26:14,814][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2_folds.train_cv.predic

[ 2020-06-22 11:26:18,297][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_0.predict)=68.40%
[ 2020-06-22 11:26:18,525][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_1.predict)=68.00%
[ 2020-06-22 11:26:18,527][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_cv.predict)=68.20%
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[ 2020-06-22 11:26:18,583][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 2_folds.

In [39]:
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of GcForest = {:.2f} %".format(acc * 100))

[ 2020-06-22 11:26:24,638][cascade_classifier.transform] X_groups_test.shape=[(268, 8)]
[ 2020-06-22 11:26:24,640][cascade_classifier.transform] group_dims=[8]
[ 2020-06-22 11:26:24,642][cascade_classifier.transform] X_test.shape=(268, 8)
[ 2020-06-22 11:26:24,643][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(268, 8)


Test Accuracy of GcForest = 80.22 %


### 深度森林测试MNIST数据集

In [56]:
def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0  # 0 or 1
    ca_config["n_cascadeRFtree"] = 1000
    ca_config["max_layers"] = 100 # 最大的层数，layer对应论文中的level
    ca_config["early_stopping_rounds"] = 3 #如果出现某层的三层以内的准确率都没有提升，层中止
    ca_config["n_classes"] = 10 #判别的类别数量
    ca_config["estimators"] = []
    
    ca_config["estimators"].append({"n_folds": 2, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 2, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 2, "type": "LogisticRegression"})
    config["cascade"] = ca_config #共使用了3个基学习器
    return config
    
config=get_toy_config()
gc = GCForest(config)
#X_train_enc是每个模型最后一层输出的结果，每一个类别的可能性
X_train_enc = gc.fit_transform(x_train, y_train)

[ 2020-06-22 13:49:41,623][cascade_classifier.fit_transform] X_groups_train.shape=[(50000, 784)],y_train.shape=(50000,),X_groups_test.shape=no_test,y_test.shape=no_test
[ 2020-06-22 13:49:41,800][cascade_classifier.fit_transform] group_dims=[784]
[ 2020-06-22 13:49:41,803][cascade_classifier.fit_transform] group_starts=[0]
[ 2020-06-22 13:49:41,818][cascade_classifier.fit_transform] group_ends=[784]
[ 2020-06-22 13:49:41,821][cascade_classifier.fit_transform] X_train.shape=(50000, 784),X_test.shape=(0, 784)
[ 2020-06-22 13:49:42,125][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(50000, 784), X_cur_test.shape=(0, 784)
[ 2020-06-22 13:49:45,336][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2_folds.train_0.predict)=93.07%
[ 2020-06-22 13:49:47,299][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2_folds.train_1.predict)=93.11%
[ 2020-06-22 13:49:47,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 2

[ 2020-06-22 13:53:20,536][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_0 - 2_folds.train_cv.predict)=95.98%
[ 2020-06-22 13:53:22,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_0.predict)=95.88%
[ 2020-06-22 13:53:25,617][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_1.predict)=95.75%
[ 2020-06-22 13:53:25,624][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 2_folds.train_cv.predict)=95.81%
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC

[ 2020-06-22 13:56:08,396][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 2_folds.train_0.predict)=95.27%
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[ 2020-06-22 13:56:24,512][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 2_folds.train_1.predict)=95.51%
[ 2020-06-22 13:56:24,519][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_2 - 2_folds.train_cv.predict)=95.39%
[ 2020-06-22 13:56:24,539][cascade_classifier.calc_accuracy] Accuracy(layer_6 - train.classifier_ave

In [57]:
y_pred = gc.predict(x_valid)
acc = accuracy_score(y_valid, y_pred)
print("Test Accuracy of GcForest = {:.2f} %".format(acc * 100))

[ 2020-06-22 14:00:43,163][cascade_classifier.transform] X_groups_test.shape=[(10000, 784)]
[ 2020-06-22 14:00:43,236][cascade_classifier.transform] group_dims=[784]
[ 2020-06-22 14:00:43,237][cascade_classifier.transform] X_test.shape=(10000, 784)
[ 2020-06-22 14:00:43,265][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 784)
[ 2020-06-22 14:00:43,936][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(10000, 814)
[ 2020-06-22 14:00:44,666][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(10000, 814)
[ 2020-06-22 14:00:45,299][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(10000, 814)


Test Accuracy of GcForest = 97.32 %
